<a href="https://colab.research.google.com/github/Connor1208/git-basics-activity/blob/main/MTH3230_Assignment3_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [207]:
import torch
import torch.nn as nn
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from typing import List
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



TICKER = "AAPL"
PERIOD = "max"
WINDOW = 30 # lookback days for features
BATCH = 64
EPOCHS = 30
LEARNING_RATE = 1e-3
HIDDEN = [128, 64] # MLP hidden sizes
P_DROPOUT = 0.1
ACTIVATION_FN = nn.ReLU
CRITERION = nn.MSELoss
OPTIMIZER = torch.optim.Adam
PATIENCE = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Create a DataFrame to display model params
params = pd.DataFrame({
    'Parameter': ['Input Window Size', 'Input Data Range', 'Hidden Layers',
                'Activation Function', 'Optimizer', 'Dropout P', 'Eval Metric',
                'Batch Size', 'Learning Rate', 'Patience'],
    'Value': [WINDOW, PERIOD, HIDDEN, ACTIVATION_FN, OPTIMIZER, P_DROPOUT, CRITERION,
            BATCH, LEARNING_RATE, PATIENCE]
})

display(params)



# Daily open/high/low/close/volume for Apple (AAPL), 1980–present.
# Forecasting task: Predict next-day closing price or return.
tick = yf.Ticker(TICKER)
hist = tick.history(period=PERIOD)
display(hist.tail())



# The input data that we want to use to predict tmr's closing value
X = pd.DataFrame(index=hist.index)
X["Close"] = hist["Close"]
X["Volume"] = hist["Volume"]
X["rolling_mean_3"] = hist["Close"].rolling(3).mean()
X["rolling_mean_14"] = hist["Close"].rolling(14).mean()
X["rolling_std_3"] = hist["Close"].rolling(3).std()
X["rolling_std_14"] = hist["Close"].rolling(14).std()
X["pct_change"] = hist["Close"].pct_change()
X = X.dropna()

# The output that we wish to predict is tmr's closing value from historical data
# The output at day i will be the closing value at day (i + 1)
y = X["Close"].shift(-1)

# Drop the last row as we don't have an output for the present day
y = y.drop(y.index[-1])
X = X.drop(X.index[-1])

# Number of features
NUM_FEATURES = X.shape[1]

print("Size of y: ", y.shape)
print("Size of X: ", X.shape)



def build_windows(X, y, window_size):
    Xw, yw = [], []

    for i in range(window_size - 1 , len(X)):

        # input: days (i - window_size + 1), (i - window_size + 2), ..., i
        Xw.append(X.values[i - window_size + 1:i + 1, :])

        # output: next-day close => close at day (i + 1) => y[i]
        yw.append(y.values[i])

    Xw = np.stack(Xw).reshape(-1, window_size * NUM_FEATURES) # (N, W*F)
    yw = np.array(yw).reshape(-1, 1) # (N, 1)

    return Xw, yw

Xw, yw = build_windows(X, y, WINDOW)




# Ref: https://medium.com/@mn05052002/building-a-simple-mlp-from-scratch-using-pytorch-7d50ca66512b
X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

# Standardize the data for training performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)





train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
val_ds = TensorDataset(X_val, y_val)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=False)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False, drop_last=False)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False, drop_last=False)

,Parameter,Value
0,Input Window Size,30
1,Input Data Range,max
2,Hidden Layers,"[128, 64]"
3,Activation Function,<class 'torch.nn.modules.activation.ReLU'>
4,Optimizer,<class 'torch.optim.adam.Adam'>
5,Dropout P,0.1
6,Eval Metric,<class 'torch.nn.modules.loss.MSELoss'>
7,Batch Size,64
8,Learning Rate,0.001
9,Patience,5


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-10-16 00:00:00-04:00,248.250000,249.039993,245.130005,247.449997,39777000,0.0,0.0
2025-10-17 00:00:00-04:00,248.020004,253.380005,247.270004,252.289993,49147000,0.0,0.0
2025-10-20 00:00:00-04:00,255.889999,264.380005,255.630005,262.239990,90483000,0.0,0.0
2025-10-21 00:00:00-04:00,261.880005,265.290009,261.829987,262.769989,46695900,0.0,0.0
2025-10-22 00:00:00-04:00,262.649994,262.850006,255.429993,258.450012,44954300,0.0,0.0


Size of y:  (11293,)
Size of X:  (11293, 7)


In [150]:
# ======== Run the same 1D-CNN for window sizes 5, 20, 100 (30 epochs each) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run the three window sizes ---
results = {}
for W in [5, 20, 100]:
    results[W] = run_window(W)

print("\nSummary (RMSE):", {W: f"{rmse:.4f}" for W, (_, rmse) in results.items()})

[W=  5] Epoch 01 | Train MSE: 10.873691 | Val MSE: 12.871126
[W=  5] Epoch 02 | Train MSE: 0.211416 | Val MSE: 5.353408
[W=  5] Epoch 03 | Train MSE: 0.129434 | Val MSE: 6.214938
[W=  5] Epoch 04 | Train MSE: 0.116763 | Val MSE: 5.933214
[W=  5] Epoch 05 | Train MSE: 0.105047 | Val MSE: 6.244199
[W=  5] Epoch 06 | Train MSE: 0.101646 | Val MSE: 10.004050
[W=  5] Epoch 07 | Train MSE: 0.099526 | Val MSE: 4.682810
[W=  5] Epoch 08 | Train MSE: 0.108315 | Val MSE: 4.867618
[W=  5] Epoch 09 | Train MSE: 0.086500 | Val MSE: 4.565565
[W=  5] Epoch 10 | Train MSE: 0.095928 | Val MSE: 4.439757
[W=  5] Epoch 11 | Train MSE: 0.090919 | Val MSE: 8.006540
[W=  5] Epoch 12 | Train MSE: 0.085079 | Val MSE: 4.409769
[W=  5] Epoch 13 | Train MSE: 0.074526 | Val MSE: 4.349424
[W=  5] Epoch 14 | Train MSE: 0.073123 | Val MSE: 5.242486
[W=  5] Epoch 15 | Train MSE: 0.078477 | Val MSE: 6.831871
[W=  5] Epoch 16 | Train MSE: 0.083380 | Val MSE: 6.932369
[W=  5] Epoch 17 | Train MSE: 0.084322 | Val MSE: 5.5

In [155]:
# lr = 0.0001
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})


[W=  5] Epoch 01 | Train MSE: 3.011993 | Val MSE: 13.423723
[W=  5] Epoch 02 | Train MSE: 0.218821 | Val MSE: 5.905666
[W=  5] Epoch 03 | Train MSE: 0.149059 | Val MSE: 55.753285
[W=  5] Epoch 04 | Train MSE: 0.270775 | Val MSE: 4.799765
[W=  5] Epoch 05 | Train MSE: 0.211173 | Val MSE: 30.502096
[W=  5] Epoch 06 | Train MSE: 0.127652 | Val MSE: 6.318776
[W=  5] Epoch 07 | Train MSE: 0.148023 | Val MSE: 23.757667
[W=  5] Epoch 08 | Train MSE: 0.170007 | Val MSE: 4.900283
[W=  5] Epoch 09 | Train MSE: 0.264191 | Val MSE: 20.572170
[W=  5] Epoch 10 | Train MSE: 0.368590 | Val MSE: 26.119159
[W=  5] Epoch 11 | Train MSE: 0.216976 | Val MSE: 16.331482
[W=  5] Epoch 12 | Train MSE: 0.144806 | Val MSE: 4.987337
[W=  5] Epoch 13 | Train MSE: 0.165448 | Val MSE: 34.236317
[W=  5] Epoch 14 | Train MSE: 0.251359 | Val MSE: 7.159016
[W=  5] Epoch 15 | Train MSE: 0.148814 | Val MSE: 4.907583
[W=  5] Epoch 16 | Train MSE: 0.234525 | Val MSE: 6.705678
[W=  5] Epoch 17 | Train MSE: 0.231265 | Val MSE

In [163]:
# lr = 0.0001
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.Tanh()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})

[W=  5] Epoch 01 | Train MSE: 3.954094 | Val MSE: 1691.410149
[W=  5] Epoch 02 | Train MSE: 0.560902 | Val MSE: 1604.428353
[W=  5] Epoch 03 | Train MSE: 0.423972 | Val MSE: 1658.676156
[W=  5] Epoch 04 | Train MSE: 0.332844 | Val MSE: 1639.542252
[W=  5] Epoch 05 | Train MSE: 0.284979 | Val MSE: 1567.443986
[W=  5] Epoch 06 | Train MSE: 0.273738 | Val MSE: 1619.024176
[W=  5] Epoch 07 | Train MSE: 0.251396 | Val MSE: 1509.468060
[W=  5] Epoch 08 | Train MSE: 0.276822 | Val MSE: 1557.777144
[W=  5] Epoch 09 | Train MSE: 0.302567 | Val MSE: 1514.999755
[W=  5] Epoch 10 | Train MSE: 0.268685 | Val MSE: 1549.580078
[W=  5] Epoch 11 | Train MSE: 0.244287 | Val MSE: 1552.336312
[W=  5] Epoch 12 | Train MSE: 0.252256 | Val MSE: 1604.886123
[W=  5] Epoch 13 | Train MSE: 0.179439 | Val MSE: 1548.025547
[W=  5] Epoch 14 | Train MSE: 0.237772 | Val MSE: 1603.377997
[W=  5] Epoch 15 | Train MSE: 0.171546 | Val MSE: 1476.560664
[W=  5] Epoch 16 | Train MSE: 0.248303 | Val MSE: 1602.360143
[W=  5] 

In [180]:
#kernel 5
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=5, padding=2)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})

[W=  5] Epoch 01 | Train MSE: 10.653061 | Val MSE: 17.825944
[W=  5] Epoch 02 | Train MSE: 0.358550 | Val MSE: 12.689399
[W=  5] Epoch 03 | Train MSE: 0.142124 | Val MSE: 10.904785
[W=  5] Epoch 04 | Train MSE: 0.136518 | Val MSE: 5.996675
[W=  5] Epoch 05 | Train MSE: 0.106497 | Val MSE: 10.736584
[W=  5] Epoch 06 | Train MSE: 0.096925 | Val MSE: 12.127007
[W=  5] Epoch 07 | Train MSE: 0.101874 | Val MSE: 4.899020
[W=  5] Epoch 08 | Train MSE: 0.105574 | Val MSE: 10.192790
[W=  5] Epoch 09 | Train MSE: 0.105982 | Val MSE: 14.784644
[W=  5] Epoch 10 | Train MSE: 0.102503 | Val MSE: 4.563679
[W=  5] Epoch 11 | Train MSE: 0.097597 | Val MSE: 6.452015
[W=  5] Epoch 12 | Train MSE: 0.110021 | Val MSE: 12.343162
[W=  5] Epoch 13 | Train MSE: 0.094375 | Val MSE: 5.556150
[W=  5] Epoch 14 | Train MSE: 0.092692 | Val MSE: 12.480950
[W=  5] Epoch 15 | Train MSE: 0.079572 | Val MSE: 4.539896
[W=  5] Epoch 16 | Train MSE: 0.090994 | Val MSE: 5.261024
[W=  5] Epoch 17 | Train MSE: 0.086586 | Val M

In [178]:
PERIOD = "max"
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.pool  = nn.AdaptiveAvgPool1d(1)   # ← Added pooling layer
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64, 1)          # ← Changed input size (64, not 64*seq_len)

        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = self.pool(x).squeeze(-1)           # ← Apply average pooling
            x = self.drop(x)
            return self.fc(x)


    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})

[W=  5] Epoch 01 | Train MSE: 15.522978 | Val MSE: 115.109838
[W=  5] Epoch 02 | Train MSE: 0.522287 | Val MSE: 11.268571
[W=  5] Epoch 03 | Train MSE: 0.314716 | Val MSE: 27.023598
[W=  5] Epoch 04 | Train MSE: 0.295885 | Val MSE: 9.399158
[W=  5] Epoch 05 | Train MSE: 0.299751 | Val MSE: 10.033674
[W=  5] Epoch 06 | Train MSE: 0.280397 | Val MSE: 22.699244
[W=  5] Epoch 07 | Train MSE: 0.282598 | Val MSE: 7.680905
[W=  5] Epoch 08 | Train MSE: 0.311038 | Val MSE: 14.862578
[W=  5] Epoch 09 | Train MSE: 0.294864 | Val MSE: 6.197982
[W=  5] Epoch 10 | Train MSE: 0.252300 | Val MSE: 6.634321
[W=  5] Epoch 11 | Train MSE: 0.301068 | Val MSE: 6.549924
[W=  5] Epoch 12 | Train MSE: 0.276179 | Val MSE: 9.475229
[W=  5] Epoch 13 | Train MSE: 0.267999 | Val MSE: 14.278427
[W=  5] Epoch 14 | Train MSE: 0.282504 | Val MSE: 12.520250
[W=  5] Epoch 15 | Train MSE: 0.249001 | Val MSE: 5.054484
[W=  5] Epoch 16 | Train MSE: 0.265138 | Val MSE: 5.312508
[W=  5] Epoch 17 | Train MSE: 0.261710 | Val M

In [179]:
PERIOD = "max"
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})

[W=  5] Epoch 01 | Train MSE: 10.962754 | Val MSE: 89.065834
[W=  5] Epoch 02 | Train MSE: 0.366383 | Val MSE: 6.880054
[W=  5] Epoch 03 | Train MSE: 0.171685 | Val MSE: 9.043439
[W=  5] Epoch 04 | Train MSE: 0.150316 | Val MSE: 6.984975
[W=  5] Epoch 05 | Train MSE: 0.129612 | Val MSE: 7.349450
[W=  5] Epoch 06 | Train MSE: 0.117366 | Val MSE: 6.361308
[W=  5] Epoch 07 | Train MSE: 0.110884 | Val MSE: 10.353462
[W=  5] Epoch 08 | Train MSE: 0.113521 | Val MSE: 7.521971
[W=  5] Epoch 09 | Train MSE: 0.109628 | Val MSE: 6.055646
[W=  5] Epoch 10 | Train MSE: 0.103785 | Val MSE: 5.747351
[W=  5] Epoch 11 | Train MSE: 0.095646 | Val MSE: 6.681102
[W=  5] Epoch 12 | Train MSE: 0.086547 | Val MSE: 9.113407
[W=  5] Epoch 13 | Train MSE: 0.093448 | Val MSE: 5.648479
[W=  5] Epoch 14 | Train MSE: 0.098443 | Val MSE: 5.488804
[W=  5] Epoch 15 | Train MSE: 0.092594 | Val MSE: 10.749243
[W=  5] Epoch 16 | Train MSE: 0.082443 | Val MSE: 9.405755
[W=  5] Epoch 17 | Train MSE: 0.107809 | Val MSE: 8.

In [201]:
#Conv64
PERIOD = "max"
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 64, kernel_size=3, padding=1)   # ← changed 32 → 64
            self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)     # ← updated input/output channels
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(128 * seq_len, 1)                      # ← updated input size (128 × seq_len)

        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)


    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})

[W=  5] Epoch 01 | Train MSE: 48706.301161 | Val MSE: 54033.847656
[W=  5] Epoch 02 | Train MSE: 48477.344088 | Val MSE: 53807.539062
[W=  5] Epoch 03 | Train MSE: 48171.894193 | Val MSE: 53491.417969
[W=  5] Epoch 04 | Train MSE: 47739.905785 | Val MSE: 53049.675781
[W=  5] Epoch 05 | Train MSE: 47142.214084 | Val MSE: 52446.925781
[W=  5] Epoch 06 | Train MSE: 46323.206989 | Val MSE: 51643.832031
[W=  5] Epoch 07 | Train MSE: 45236.809713 | Val MSE: 50599.582031
[W=  5] Epoch 08 | Train MSE: 43816.969975 | Val MSE: 49266.675781
[W=  5] Epoch 09 | Train MSE: 42004.051056 | Val MSE: 47597.191406
[W=  5] Epoch 10 | Train MSE: 39788.418940 | Val MSE: 45541.652344
[W=  5] Epoch 11 | Train MSE: 37117.104012 | Val MSE: 43055.058594
[W=  5] Epoch 12 | Train MSE: 33900.350591 | Val MSE: 40106.964844
[W=  5] Epoch 13 | Train MSE: 30209.531503 | Val MSE: 36701.191406
[W=  5] Epoch 14 | Train MSE: 26155.581482 | Val MSE: 32848.023438
[W=  5] Epoch 15 | Train MSE: 21667.893877 | Val MSE: 28623.98

In [194]:
#depth 3
PERIOD = "max"
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Conv1d(64->128)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)   # ← added third conv layer
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(128 * seq_len, 1)                   # ← updated input size

        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = self.act(self.conv3(x))                                # ← added forward pass for conv3
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    # Training setup
    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Evaluation function
    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train 30 epochs
    for epoch in range(1, 31):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            tr_tot += loss.item() * xb.size(0)
            tr_n += xb.size(0)
        train_mse = tr_tot / tr_n
        val_mse = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)
print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})


[W=  5] Epoch 01 | Train MSE: 6.148822 | Val MSE: 6.864428
[W=  5] Epoch 02 | Train MSE: 0.123506 | Val MSE: 10.159603
[W=  5] Epoch 03 | Train MSE: 0.103401 | Val MSE: 8.721442
[W=  5] Epoch 04 | Train MSE: 0.087203 | Val MSE: 6.857355
[W=  5] Epoch 05 | Train MSE: 0.079216 | Val MSE: 8.915024
[W=  5] Epoch 06 | Train MSE: 0.082226 | Val MSE: 12.288895
[W=  5] Epoch 07 | Train MSE: 0.081886 | Val MSE: 5.687261
[W=  5] Epoch 08 | Train MSE: 0.085390 | Val MSE: 7.303789
[W=  5] Epoch 09 | Train MSE: 0.073529 | Val MSE: 5.571581
[W=  5] Epoch 10 | Train MSE: 0.085984 | Val MSE: 4.659429
[W=  5] Epoch 11 | Train MSE: 0.074834 | Val MSE: 4.789211
[W=  5] Epoch 12 | Train MSE: 0.079135 | Val MSE: 6.188665
[W=  5] Epoch 13 | Train MSE: 0.073605 | Val MSE: 5.038219
[W=  5] Epoch 14 | Train MSE: 0.076690 | Val MSE: 7.050550
[W=  5] Epoch 15 | Train MSE: 0.061881 | Val MSE: 21.856019
[W=  5] Epoch 16 | Train MSE: 0.086404 | Val MSE: 7.637629
[W=  5] Epoch 17 | Train MSE: 0.076236 | Val MSE: 19.

In [212]:
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (same layers; regularization change: NO dropout)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Identity()              # <-- removed dropout (was Dropout(0.1))
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()

    # Regularization: add **L2 weight decay**; keep optimizer and LR the same
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)
print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})



[W=  5] Epoch 01 | Train MSE: 2.297120 | Val MSE: 8.992718
[W=  5] Epoch 02 | Train MSE: 0.082944 | Val MSE: 5.310494
[W=  5] Epoch 03 | Train MSE: 0.057668 | Val MSE: 8.748298
[W=  5] Epoch 04 | Train MSE: 0.048098 | Val MSE: 5.309777
[W=  5] Epoch 05 | Train MSE: 0.054679 | Val MSE: 10.563286
[W=  5] Epoch 06 | Train MSE: 0.046912 | Val MSE: 7.959372
[W=  5] Epoch 07 | Train MSE: 0.056324 | Val MSE: 142.722487
[W=  5] Epoch 08 | Train MSE: 1.078042 | Val MSE: 6.359568
[W=  5] Epoch 09 | Train MSE: 0.041402 | Val MSE: 4.962978
[W=  5] Epoch 10 | Train MSE: 0.034450 | Val MSE: 4.455926
[W=  5] Epoch 11 | Train MSE: 0.041340 | Val MSE: 4.664433
[W=  5] Epoch 12 | Train MSE: 0.034837 | Val MSE: 5.065785
[W=  5] Epoch 13 | Train MSE: 0.035912 | Val MSE: 5.412027
[W=  5] Epoch 14 | Train MSE: 0.031085 | Val MSE: 5.090481
[W=  5] Epoch 15 | Train MSE: 0.031303 | Val MSE: 7.784324
[W=  5] Epoch 16 | Train MSE: 0.053154 | Val MSE: 40.450616
[W=  5] Epoch 17 | Train MSE: 0.143842 | Val MSE: 30

In [199]:
# ======== 1D-CNN (window=5) using 1 year of data (30 epochs) ========
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- Data (1 year) ----------------
TICKER = "AAPL"
PERIOD = "1y"  # <-- one year
tick = yf.Ticker(TICKER)
hist = tick.history(period=PERIOD)
display(hist.tail())

# Feature table X (7 features) and next-day target y
X = pd.DataFrame(index=hist.index)
X["Close"]           = hist["Close"]
X["Volume"]          = hist["Volume"]
X["rolling_mean_3"]  = hist["Close"].rolling(3).mean()
X["rolling_mean_14"] = hist["Close"].rolling(14).mean()
X["rolling_std_3"]   = hist["Close"].rolling(3).std()
X["rolling_std_14"]  = hist["Close"].rolling(14).std()
X["pct_change"]      = hist["Close"].pct_change()
X = X.dropna()

y = X["Close"].shift(-1)      # next-day Close
y = y.drop(y.index[-1])
X = X.drop(X.index[-1])

NUM_FEATURES = X.shape[1]
print("Size of X:", X.shape, "| Size of y:", y.shape)

# ---------------- Sliding windows (stride = 1 day) ----------------
def build_windows(X_df: pd.DataFrame, y_sr: pd.Series, window_size: int):
    Xv, yv = X_df.values, y_sr.values
    W = window_size
    xs, ys = [], []
    # pair each window X[i-W+1:i+1] with y[i]  (y is already next-day close)
    for i in range(W - 1, len(Xv)):
        xs.append(Xv[i - W + 1 : i + 1, :])   # (W, F)
        ys.append(yv[i])                       # scalar
    Xw = np.stack(xs).reshape(-1, W * NUM_FEATURES).astype(np.float32)
    yw = np.array(ys, dtype=np.float32).reshape(-1, 1)
    return Xw, yw

# ---------------- Runner (window=5) ----------------
def run_window(window_size: int):
    # Rebuild windows for this size
    Xw, yw = build_windows(X, y, window_size)

    # Chronological 80/10/10 split
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val,   X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model: Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->



    ERIOD = "1m"
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[5] = run_window(5)

print("\nSummary (RMSE):", {5: f"{results[5][1]:.4f}"})


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-10-16 00:00:00-04:00,248.250000,249.039993,245.130005,247.449997,39777000,0.0,0.0
2025-10-17 00:00:00-04:00,248.020004,253.380005,247.270004,252.289993,49147000,0.0,0.0
2025-10-20 00:00:00-04:00,255.889999,264.380005,255.630005,262.239990,90483000,0.0,0.0
2025-10-21 00:00:00-04:00,261.880005,265.290009,261.829987,262.769989,46695900,0.0,0.0
2025-10-22 00:00:00-04:00,262.649994,262.850006,255.429993,258.450012,44954300,0.0,0.0


Size of X: (236, 7) | Size of y: (236,)
[W=  5] Epoch 01 | Train MSE: 48818.752111 | Val MSE: 54206.800781
[W=  5] Epoch 02 | Train MSE: 48725.828252 | Val MSE: 54122.476562
[W=  5] Epoch 03 | Train MSE: 48626.929645 | Val MSE: 54031.406250
[W=  5] Epoch 04 | Train MSE: 48517.298670 | Val MSE: 53924.820312
[W=  5] Epoch 05 | Train MSE: 48386.315203 | Val MSE: 53791.878906
[W=  5] Epoch 06 | Train MSE: 48219.583742 | Val MSE: 53622.097656
[W=  5] Epoch 07 | Train MSE: 48005.718159 | Val MSE: 53401.300781
[W=  5] Epoch 08 | Train MSE: 47723.877154 | Val MSE: 53118.625000
[W=  5] Epoch 09 | Train MSE: 47373.437394 | Val MSE: 52761.332031
[W=  5] Epoch 10 | Train MSE: 46917.433974 | Val MSE: 52316.152344
[W=  5] Epoch 11 | Train MSE: 46354.761043 | Val MSE: 51765.500000
[W=  5] Epoch 12 | Train MSE: 45665.298860 | Val MSE: 51089.875000
[W=  5] Epoch 13 | Train MSE: 44817.247530 | Val MSE: 50276.460938
[W=  5] Epoch 14 | Train MSE: 43803.561951 | Val MSE: 49305.273438
[W=  5] Epoch 15 | Tra

In [232]:
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[7] = run_window(7)

print("\nSummary (RMSE):", {7: f"{results[7][1]:.4f}"})

[W=  7] Epoch 01 | Train MSE: 6.682632 | Val MSE: 41.642305
[W=  7] Epoch 02 | Train MSE: 0.229277 | Val MSE: 13.472077
[W=  7] Epoch 03 | Train MSE: 0.136638 | Val MSE: 11.524025
[W=  7] Epoch 04 | Train MSE: 0.122124 | Val MSE: 9.984999
[W=  7] Epoch 05 | Train MSE: 0.105437 | Val MSE: 14.691109
[W=  7] Epoch 06 | Train MSE: 0.108262 | Val MSE: 5.664286
[W=  7] Epoch 07 | Train MSE: 0.097827 | Val MSE: 5.310065
[W=  7] Epoch 08 | Train MSE: 0.089626 | Val MSE: 5.529080
[W=  7] Epoch 09 | Train MSE: 0.080191 | Val MSE: 8.463592
[W=  7] Epoch 10 | Train MSE: 0.089235 | Val MSE: 6.070410
[W=  7] Epoch 11 | Train MSE: 0.091739 | Val MSE: 10.319981
[W=  7] Epoch 12 | Train MSE: 0.072355 | Val MSE: 8.421638
[W=  7] Epoch 13 | Train MSE: 0.073578 | Val MSE: 11.051173
[W=  7] Epoch 14 | Train MSE: 0.076960 | Val MSE: 5.884794
[W=  7] Epoch 15 | Train MSE: 0.087560 | Val MSE: 6.323450
[W=  7] Epoch 16 | Train MSE: 0.066592 | Val MSE: 5.846102
[W=  7] Epoch 17 | Train MSE: 0.069589 | Val MSE: 

In [234]:
#Stride with window 7
def build_windows_multi_stride(X_df, y_sr, window_size=7):
    """
    Build overlapping weekly blocks with 7 offset phases.
    For each offset in [0..6]:
      (1) start at offset
      (2) step forward by window_size
      (3) collect (window_size)-length blocks
    Produces all patterns like (1–7), (2–8), ... up to (~T-7).
    """
    F = X_df.shape[1]
    Xv, yv = X_df.values, y_sr.values
    T = len(Xv)

    xs, ys = [], []

    for offset in range(window_size):   # e.g. 0→6
        for start in range(offset, T - window_size, window_size):
            end = start + window_size
            if end >= len(yv):
                break
            xs.append(Xv[start:end, :])   # window (7,F)
            ys.append(yv[end])            # next-day close after window

    Xw = np.stack(xs).reshape(-1, window_size * F).astype(np.float32)
    yw = np.array(ys, dtype=np.float32).reshape(-1, 1)
    return Xw, yw


In [237]:
#implemented build_windows_multi_stride
# ======== Run the same 1D-CNN but only for window size 5 (30 epochs) ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_window(window_size: int):
    # --- Rebuild windows for this size (uses your existing build_windows) ---
    Xw, yw = build_windows_multi_stride(X, y, window_size)

    # Splits (chronological)
    X_train, X_tmp, y_train, y_tmp = train_test_split(Xw, yw, test_size=0.2, shuffle=False)
    X_val, X_test, y_val, y_test   = train_test_split(X_tmp, y_tmp, test_size=0.5, shuffle=False)

    # Scale (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # Tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_val   = torch.tensor(X_val,   dtype=torch.float32)
    y_val   = torch.tensor(y_val,   dtype=torch.float32).reshape(-1, 1)
    X_test  = torch.tensor(X_test,  dtype=torch.float32)
    y_test  = torch.tensor(y_test,  dtype=torch.float32).reshape(-1, 1)

    # Loaders
    train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True,  drop_last=False)
    val_dl   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=64, shuffle=False, drop_last=False)
    test_dl  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False, drop_last=False)

    # Model (Conv1d(F->32)->ReLU->Conv1d(32->64)->ReLU->Flatten->Dropout(0.1)->Linear->1)
    class CNNRegressor(nn.Module):
        def __init__(self, in_ch=NUM_FEATURES, seq_len=window_size):
            super().__init__()
            self.conv1 = nn.Conv1d(in_ch, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
            self.act   = nn.ReLU()
            self.drop  = nn.Dropout(0.1)
            self.fc    = nn.Linear(64 * seq_len, 1)
        def forward(self, x: Tensor) -> Tensor:
            x = self.act(self.conv1(x))
            x = self.act(self.conv2(x))
            x = torch.flatten(x, 1)
            x = self.drop(x)
            return self.fc(x)

    model = CNNRegressor().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    def eval_mse(dataloader):
        model.eval()
        tot, n = 0.0, 0
        with torch.no_grad():
            for xb, yb in dataloader:
                xb, yb = xb.to(device), yb.to(device)
                xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)
                pred = model(xb)
                tot += criterion(pred, yb).item() * xb.size(0)
                n += xb.size(0)
        return tot / max(n, 1)

    # Train exactly 30 epochs
    for epoch in range(1, 30 + 1):
        model.train()
        tr_tot, tr_n = 0.0, 0
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.reshape(-1, window_size, NUM_FEATURES).permute(0, 2, 1)

            optimizer.zero_grad(set_to_none=True)
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

            tr_tot += loss.item() * xb.size(0)
            tr_n   += xb.size(0)

        train_mse = tr_tot / tr_n
        val_mse   = eval_mse(val_dl)
        print(f"[W={window_size:3d}] Epoch {epoch:02d} | Train MSE: {train_mse:.6f} | Val MSE: {val_mse:.6f}")

    # Final test
    test_mse = eval_mse(test_dl)
    test_rmse = float(np.sqrt(test_mse))
    print(f"[W={window_size:3d}] Test MSE: {test_mse:.6f} | Test RMSE: {test_rmse:.6f}")
    return test_mse, test_rmse

# --- Run only window size = 5 ---
results = {}
results[7] = run_window(7)

print("\nSummary (RMSE):", {7: f"{results[7][1]:.4f}"})

[W=  7] Epoch 01 | Train MSE: 164.366388 | Val MSE: 10.910196
[W=  7] Epoch 02 | Train MSE: 7.301999 | Val MSE: 37.008372
[W=  7] Epoch 03 | Train MSE: 7.416757 | Val MSE: 17.310803
[W=  7] Epoch 04 | Train MSE: 10.510647 | Val MSE: 23.452052
[W=  7] Epoch 05 | Train MSE: 18.469767 | Val MSE: 7.276527
[W=  7] Epoch 06 | Train MSE: 9.270459 | Val MSE: 10.952903
[W=  7] Epoch 07 | Train MSE: 7.498492 | Val MSE: 112.587561
[W=  7] Epoch 08 | Train MSE: 9.865797 | Val MSE: 20.226877
[W=  7] Epoch 09 | Train MSE: 9.623229 | Val MSE: 9.076322
[W=  7] Epoch 10 | Train MSE: 7.101661 | Val MSE: 6.838195
[W=  7] Epoch 11 | Train MSE: 9.600896 | Val MSE: 5.484982
[W=  7] Epoch 12 | Train MSE: 11.082756 | Val MSE: 450.190036
[W=  7] Epoch 13 | Train MSE: 19.654847 | Val MSE: 5.823317
[W=  7] Epoch 14 | Train MSE: 10.826247 | Val MSE: 44.299647
[W=  7] Epoch 15 | Train MSE: 7.642727 | Val MSE: 5.764882
[W=  7] Epoch 16 | Train MSE: 8.350522 | Val MSE: 33.018065
[W=  7] Epoch 17 | Train MSE: 12.9792